In [ ]:
from flask import Flask, render_template, request, jsonify, url_for
import random
import json
import torch
from model import NeuralNet
from nltk_utils import bag_of_words, tokenize

app = Flask(__name__)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Sam"
"""print("Let's chat! (type 'quit' to exit)")"""

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/chat", methods=["POST"])
def chat():
    user_message = request.form["user_message"]
    if user_message.lower() == "quit":
        #return render_template("index.html", response="You have quit the chat.")
        return "You have to quit the chat"
    sentence = tokenize(user_message)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                response = random.choice(intent['responses'])
                #return jsonify({"response": f"{bot_name}: {response}"})
                return jsonify({"response": f"{response}"})
                
               
    else:
        response = "I do not understand..."
        #return jsonify({"response": f"{bot_name}: {response}"})
        return jsonify({"response": f"{response}"})
    #return render_template("index.html", response=f"{bot_name}: {response}")

if __name__ == "__main__":
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Apr/2024 20:12:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2024 20:12:39] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2024 20:12:39] "GET /static/script.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2024 20:12:39] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [26/Apr/2024 20:12:43] "POST /chat HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2024 20:12:49] "POST /chat HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2024 20:15:58] "POST /chat HTTP/1.1" 200 -
